In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [2]:
# Load the training set
df = pd.read_csv(r'.\gamified_dataset/Dyt-desktop.csv')

print("#-----> First 5 rows of the training set:\n")
df.head(5)


#-----> First 5 rows of the training set:



,Gender,Nativelang,Otherlang,Age,Clicks1,Hits1,Misses1,Score1,Accuracy1,Missrate1,...,Score31,Accuracy31,Missrate31,Clicks32,Hits32,Misses32,Score32,Accuracy32,Missrate32,Dyslexia
0,Male,No,Yes,7,10,10,0,10,1.0,0.0,...,0,0.000000,0.00,17,2,0,2,0.117647,0.000000,No
1,Female,Yes,Yes,13,12,12,0,12,1.0,0.0,...,4,0.114286,0.00,26,2,2,2,0.076923,0.076923,Yes
2,Female,No,Yes,7,6,6,0,6,1.0,0.0,...,4,0.114286,0.00,26,1,3,1,0.038462,0.115385,No
3,Female,No,Yes,7,0,0,0,0,0.0,0.0,...,0,0.000000,0.00,1,0,0,0,0.000000,0.000000,No
4,Female,No,Yes,8,4,4,0,4,1.0,0.0,...,1,25.000000,0.05,26,2,2,2,0.076923,0.076923,No


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [4]:
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['Otherlang'] = label_encoder.fit_transform(df['Otherlang'])
df['Nativelang'] = label_encoder.fit_transform(df['Nativelang'])
df['Dyslexia'] = label_encoder.fit_transform(df['Dyslexia'])

In [5]:
X_features = df.drop(['Dyslexia'], axis=1)
y_labels= df['Dyslexia']

In [6]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.2, random_state=42, stratify=y_labels)

# Standardize the input features (optional but recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.2, random_state=42, stratify=y_labels)

# Standardize the input features (optional but recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [8]:
from sklearn.metrics import recall_score,classification_report

# Calculate class weights
class_weights = {0: 1, 1: 2}  # Adjust the weight for dyslexia class (1) according to your preference

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=1, input_dim = X_train.shape[1]),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

# Compile the model with additional metrics
model.compile(optimizer='adam',
              loss='binary_crossentropy')

# Cross-validate the model with the custom scorer and different decision thresholds
thresholds = np.arange(0.2, 0.3, 0.005)
for threshold in thresholds:
    model_history = model.fit(X_train, y_train, epochs=10,verbose=0, 
                              class_weight=class_weights)
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > threshold).astype(int)
    
    # Evaluate recall on the test set with the current threshold
    recall = recall_score(y_test, y_pred)
    
    # Print the current threshold and recall
    print(f"Threshold: {threshold:.3f}, Recall: {recall:.4f}")
    classification_rep_test = classification_report(y_test, y_pred)
    print(classification_rep_test)





23/23 [==============================] - 0s 1ms/step
Threshold: 0.200, Recall: 0.7051
              precision    recall  f1-score   support

           0       0.95      0.69      0.80       651
           1       0.22      0.71      0.33        78

    accuracy                           0.70       729
   macro avg       0.58      0.70      0.57       729
weighted avg       0.87      0.70      0.75       729

23/23 [==============================] - 0s 1ms/step
Threshold: 0.205, Recall: 0.7564
              precision    recall  f1-score   support

           0       0.96      0.71      0.82       651
           1       0.24      0.76      0.36        78

    accuracy                           0.72       729
   macro avg       0.60      0.73      0.59       729
weighted avg       0.88      0.72      0.77       729

23/23 [==============================] - 0s 1ms/step
Threshold: 0.210, Recall: 0.7821
              precision    recall  f1-score   support

           0       0.97      0

In [10]:
threshold = 0.265
model_history = model.fit(X_train, y_train, epochs=10,verbose=0, 
                          class_weight=class_weights)
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > threshold).astype(int)

# Evaluate recall on the test set with the current threshold
recall = recall_score(y_test, y_pred)

# Print the current threshold and recall
print(f"Threshold: {threshold:.3f}, Recall: {recall:.4f}")
classification_rep_test = classification_report(y_test, y_pred)
print(classification_rep_test)


converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_converted_model = converter.convert()

with open('lr_model.tflite', 'wb') as f:
  f.write(tflite_converted_model)

23/23 [==============================] - 0s 1ms/step
Threshold: 0.265, Recall: 0.7179
              precision    recall  f1-score   support

           0       0.96      0.84      0.90       651
           1       0.36      0.72      0.48        78

    accuracy                           0.83       729
   macro avg       0.66      0.78      0.69       729
weighted avg       0.90      0.83      0.85       729

INFO:tensorflow:Assets written to: C:\Users\Roaa\AppData\Local\Temp\tmpnf4uy11z\assets


INFO:tensorflow:Assets written to: C:\Users\Roaa\AppData\Local\Temp\tmpnf4uy11z\assets
